In [ ]:
%matplotlib notebook

import numpy as np
import pylab as plt
plt.rc('text', usetex=True)
plt.rc('font', size=18,family='serif')
import matplotlib.gridspec as gridspec
from emcee import EnsembleSampler
from emcee.backends import HDFBackend
from multiprocessing import cpu_count, Pool
from corner import corner
from Cosmology import *
import sys


#load autoreload, which automatically reloads the cosmology.py upon execution
%load_ext autoreload
%autoreload 1
%aimport Cosmology


#Parameters for BAO -- CHECK THESE VALUES
z_d = 1089
h = .6727
omega_baryon_preset = 0.02236/h**2
omega_gamma_preset = 2.469E-5/h**2

#Cosmological parameters
Omega_m_preset = 0.3166
Omega_r_preset = 2.469E-5/h**2
Omega_c_preset = 0.6834

# Import data

## Supernova data

In [ ]:
dataSN = np.loadtxt('data/jla_lcparams.txt', usecols=(2,4,6,8,10))
errSN = np.loadtxt('data/jla_lcparams.txt', usecols=(5,7,9))[np.argsort(dataSN.T[0])]
dataSN = dataSN[np.argsort(dataSN.T[0])]

#best fit values found in JLA analysis arXiv:1401.4064
a = 0.14
b = 3.14
MB = -19.04
delta_Mhost = -.06

In [ ]:
SNdata = Supernova_data(dataSN, errSN, np.array([a,b,MB, 0]))

## Quasar data

In [ ]:
dataQ = np.loadtxt('data/quasar_data_RL.txt', usecols=(0,1,2,3,4))
errQ = np.loadtxt('data/quasar_data_RL.txt', usecols=5)[np.argsort(dataQ.T[0])]
dataQ = dataQ[np.argsort(dataQ.T[0])]

#best fit values found in Risaliti & Lusso, Nature Astronomy, 2018
beta_prime, s = 7.4, 1.5

In [ ]:
Qdata = Quasar_data(dataQ, errQ, np.array([beta_prime, s]))

# BAO data

In [ ]:
#load BOSS data points
dataBAO = np.loadtxt('data/BOSS.txt', usecols=(1,3))
#load BOSS errors & measurement quantity
errBAO  = np.diag(np.loadtxt('data/BOSS.txt', usecols=4))
typeBAO = np.genfromtxt('data/BOSS.txt',dtype=str, usecols=2)

#BOSS DR12 covariance matrix from 1607.03155
sigmaDR12 = np.diag(np.diag(errBAO)[np.where(np.loadtxt('data/BOSS.txt', usecols=0, dtype=str) == 'BOSS_DR12')])
corrDR12 = np.tril(np.loadtxt('data/BOSS_DR12_cov.txt', usecols=(1,2,3,4,5,6))*1E-4)
corrDR12 += corrDR12.T
corrDR12 /= 2
CovDR12 = np.dot(sigmaDR12, np.dot(corrDR12, sigmaDR12))

#eBOSS Quasar covariance matrix from 1801.03043
sigmaeBOSS = np.diag(np.diag(errBAO)[np.where(np.loadtxt('data/BOSS.txt', usecols=0, dtype=str) == 'eBOSS_QSO')])
correBOSS = np.triu(np.loadtxt('data/eBOSS_QSO_cov.txt', usecols=(1,2,3,4,5,6,7,8))*1E-4)
correBOSS += correBOSS.T
correBOSS /= 2
CoveBOSS = np.dot(sigmaeBOSS, np.dot(correBOSS, sigmaeBOSS))

#assemble the full covariance matrix
load_cov = np.diag([1 if i else 0 for i in np.loadtxt('data/BOSS.txt',usecols=5)==1]) 
CovBAO = (errBAO - np.dot(load_cov,errBAO))**2
CovBAO += np.pad(CovDR12,[(2,len(errBAO)-2-len(CovDR12)),(2,len(errBAO)-2-len(CovDR12))], mode='constant', constant_values=0)
CovBAO += np.pad(CoveBOSS,[(2 + len(CovDR12) + 1,len(errBAO)- 3 - len(CovDR12) - len(CoveBOSS)),(2 + len(CovDR12) + 1,len(errBAO)- 3 - len(CovDR12) - len(CoveBOSS))], mode='constant', constant_values=0)

#Finally, add the correlations given in 1904.03430 
add_cov = np.array([i if i<0 else 0 for i in np.loadtxt('data/BOSS.txt',usecols=5)])
Cov = np.pad(np.diag(add_cov[1:]), [(1,0),(0,1)], mode='constant', constant_values=0) + np.pad(np.diag(add_cov[1:]), [(0,1), (1,0)], mode='constant', constant_values=0)
for ind in np.array([np.where(add_cov<0)[0]-1., np.where(add_cov<0)[0]], dtype=int).T:
    Cov[ind] = Cov[ind] * np.diag(errBAO)[ind[0]] * np.diag(errBAO)[ind[1]]
    
CovBAO += Cov

BAOdata = BAO_data(dataBAO, CovBAO, typeBAO)
LCDM = cosmology(Omega_m_preset, Omega_c_preset, omegab=omega_baryon_preset, omegar=omega_gamma_preset, w=-1, Hzero=100.*h)

In [ ]:
# # load all data points except for WiggleZ
# dataBAO = np.loadtxt('data/BOSS.txt', usecols=(1,3))
# # dataBAO = np.loadtxt('data/bao_1806-06781.txt', usecols=(1,3))
# # add WiggleZ
# dataBAO = np.append(dataBAO, np.loadtxt('data/WiggleZ.txt', usecols=(1,3)), axis=0)

# # the error for BAO is a cov mat, due to the addition of the WiggleZ data.
# errBAO  = np.pad(np.diag(np.loadtxt('data/BOSS.txt', usecols=4)), [(0, 3), (0, 3)], mode='constant', constant_values=0)
# # errBAO  = np.pad(np.diag(np.loadtxt('data/bao_1806-06781.txt', usecols=4)), [(0, 3), (0, 3)], mode='constant', constant_values=0)

# # now add the WiggleZ cov mat. Note that this is the sqrt, as all the other errors are given in this format, too.
# errBAO += np.pad(np.sqrt(np.loadtxt('data/WiggleZ_cov.txt', usecols=(3,4,5))), [(len(errBAO)-3, 0), (len(errBAO)-3, 0)], mode='constant', constant_values=0)

# typeBAO = np.genfromtxt('data/BOSS.txt',dtype=str, usecols=2)
# # typeBAO = np.genfromtxt('data/bao_1806-06781.txt',dtype=str, usecols=2)
# typeBAO = np.append(typeBAO, np.genfromtxt('data/WiggleZ.txt',dtype=str, usecols=2), axis=0)

In [ ]:
# BAOdata = BAO_data(dataBAO,errBAO**2, np.array([omega_baryon_preset, omega_gamma_preset]), typeBAO)
# LCDM = cosmology(Omega_m_preset, Omega_c_preset, w=-1, Hzero=69)

# RC data

In [ ]:
gamma00, kappa0 = 0.0093, 95
RCdata = RC_data([gamma00, kappa0])

# Planck 2018 CMB data

In [ ]:
CMBdata = CMB_data('Planck18')

# Model 0: $\Lambda$LCDM

## which data sets to include?

In [ ]:
data_sets = [SNdata, CMBdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

## Define the likelihood and priors

In [ ]:
ranges_LCDM_min=[0, 0, 60., -5, -10, -30, -.5, 0, 0]
ranges_LCDM_max=[1, 0.1, 80., 5, 10, -10, .5, 10, 3]
def lnprob_LCDM(theta):
    l = likelihood(theta, data_sets, ranges_LCDM_min, ranges_LCDM_max, model = 'LCDM')
    return l.logprobability_gauss_prior()

## define a reference BIC

In [ ]:
theta_LCDM = [Omega_m_preset, omega_baryon_preset, 67.27, a, b, MB, -0.05, beta_prime, s]

BIC_LCDM = lambda theta: np.log(sum([len(d.get_data()) for d in data_sets])) * len(theta) -2*lnprob_LCDM(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = 9, 50, 100
pos0 = np.random.uniform(ranges_LCDM_min, ranges_LCDM_max, (nwalkers,len(ranges_LCDM_max)))

# pool = Pool(cpu_count())
# write = HDFBackend('chains/LCDM_' + str(nwalkers) + 'x' + str(nsteps) + '.h5')
# sampler = EnsembleSampler(nwalkers, ndim, lnprob_LCDM, pool=pool)#, backend=write)

# sampler.run_mcmc(pos0, nsteps, progress=True)

## ...or load existing chains

In [ ]:
LCDM_sampler = HDFBackend('chains/LCDM' + data_sets_str + '512x1000.h5', read_only=True)
# LCDM_sampler = sampler

nsteps, nwalkers, ndim = LCDM_sampler.get_chain().shape

labs = [r'$\Omega_m$', r'$\Omega_b$', r'$H_0$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']

## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 300
while Nmin < nsteps and np.all(LCDM_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
LCDM_samples = LCDM_sampler.get_chain()

f, ax = plt.subplots(len(LCDM_samples.T), 1, figsize=(6,3*len(LCDM_samples.T)))

for i in range(len(LCDM_samples.T)):
    ax[i].plot(LCDM_samples.T[i].T, lw=0.5)
#     ax[i].plot([Nmin, Nmin], [min(LCDM_samples.T[i].flatten()), max(LCDM_samples.T[i].flatten())], c='k', lw=1.5)
#     ax[i].text(Nmin, min(LCDM_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
# plt.savefig('plots/convergence_LCDM.png')
plt.show()

## Plot the 1$\sigma$ and 2$\sigma$ contours

In [ ]:
LCDM_samples = LCDM_samples[Nmin:, :].reshape((-1, ndim))


meanLCDM = np.mean(LCDM_samples, axis=0)
stdLCDM = np.var(LCDM_samples, axis=0)
maxLCDM=[]
for i in range(len(meanLCDM)):
    like = np.histogram(LCDM_samples.T[i], bins=50)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxLCDM.append(max_val)

fig = corner(LCDM_samples[:,:3], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),# range=[(0,0.8),(0,1)],
             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
#              truths=maxLCDM)




plt.tight_layout()
# plt.savefig('plots/posterior_LCDM' + data_sets_str[:-1] + '.pdf')

## Visualize the results

In [ ]:
v = np.percentile(LCDM_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

omegam, stdmp, stdmm = v[0]

omegab, stdbp, stdbm = v[1]
H0, stdhp, stdhm = v[2]
    
    
#model
z = SNdata.get_data().T[0]
best_fit_cosmo_LCDM = cosmology(omegam=maxLCDM[0],omegac=1-maxLCDM[0], omegab = maxLCDM[1], omegar=omega_gamma_preset, Hzero=maxLCDM[2])
SNdata.set_param(v.T[0][3:7])
Qdata.set_param(v.T[0][7:])

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])

if 'BAO'  in data_set_names or 'CMB' in data_set_names:    
    ax1.set_title('$\Omega_m = {%1.3f}^{+%1.3f}_{-%1.3f},\, H_0 = {%1.1f}^{+%1.1f}_{-%1.1f}$' % (omegam, stdmp,stdmm, H0, stdhm, stdhp))
else:
    ax1.set_title('$\Omega_m = {%1.3f}^{+%1.3f}_{-%1.3f}$' % (omegam, stdmp,stdmm))
    
if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=0, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(best_fit_cosmo_LCDM).T[0], BAOdata.distance_modulus(best_fit_cosmo_LCDM).T[1], yerr=BAOdata.delta_distance_modulus(best_fit_cosmo_LCDM), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=1, label=r'BAO')



zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_cosmo_LCDM.distance_modulus(zPlot), c='k', label=r'fit')
ax1.fill_between(zPlot, cosmology(omegam+stdmp,1-omegam+stdmm, omegab=omegab+stdbp, Hzero=H0+stdhp).distance_modulus(zPlot), cosmology(omegam-stdmm,1-omegam-stdmp, omegab=omegab-stdbm, Hzero=H0-stdhm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend()


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_cosmo_LCDM.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .94, r'$\Delta \mathrm{BIC}(\Lambda\mathrm{CDM}) = %1.0f$' % (BIC_LCDM(maxLCDM) - BIC_LCDM(theta_LCDM)), ha='right', va='top', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
# plt.savefig('plots/Hubble_LCDM' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

In [ ]:
LCDM_chains = {}
LCDM_chains['SN'] = HDFBackend('chains/LCDM_SN_512x1000.h5')
LCDM_chains['SN_Q'] = HDFBackend('chains/LCDM_SN_Quasars_512x1000.h5')
LCDM_chains['SN_Q_BAO'] = HDFBackend('chains/LCDM_SN_Quasars_BAO_512x1000.h5')
LCDM_chains['BAO'] = HDFBackend('chains/LCDM_BAO_512x1000.h5')

In [ ]:
Nmin = 300
for sampler in LCDM_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print('Nmin=',Nmin)

In [ ]:
samples = LCDM_chains['BAO'].get_chain()[Nmin:,:].reshape((-1, LCDM_chains['SN_Q_BAO'].get_chain().shape[-1]))
fig=corner(samples[:,:2], quantiles=(.16,.84),  levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgray', 'gray'], alpha=0.4),
            range=[(0,.6), (62,75)])


# samples = LCDM_chains['SN'].get_chain()[Nmin:,:].reshape((-1, LCDM_chains['SN'].get_chain().shape[-1]))
# corner(samples[:,:2], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
#             fig=fig, range=[(0,.6), (62,75)])

# samples = LCDM_chains['SN_Q'].get_chain()[Nmin:,:].reshape((-1, LCDM_chains['SN_Q'].get_chain().shape[-1]))
# corner(samples[:,:2], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
#             fig=fig, range=[(0,.6), (62,75)])


samples = LCDM_chains['SN_Q_BAO'].get_chain()[Nmin:,:].reshape((-1, LCDM_chains['SN_Q_BAO'].get_chain().shape[-1]))
corner(samples[:,:2], quantiles=(.16,.84),  levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig, range=[(0,.6), (62,75)])



plt.tight_layout()
# plt.savefig('plots/posterior_LCDM.pdf')

plt.show()

# Model 1: o$\Lambda$LCDM

## which data sets to include?

In [ ]:
data_sets = [SNdata, Qdata, BAOdata, CMBdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

## Define the likelihood and priors

In [ ]:
ranges_oLCDM_min=[0, 0, 0, 60., -5, -10, -30, -.5, 0, 0]
ranges_oLCDM_max=[1, 1.5, .1, 80., 5, 10, -10, .5, 10, 3]
def lnprob_oLCDM(theta):
    l = likelihood(theta, data_sets, ranges_oLCDM_min, ranges_oLCDM_max, model = 'oLCDM')
    return l.logprobability_flat_prior()

## define a reference BIC

In [ ]:
theta_oLCDM = [Omega_m_preset, Omega_c_preset, omega_baryon_preset, 67.27, a, b, MB, -0.05, beta_prime, s]

BIC_oLCDM = lambda theta: np.log(sum([len(d.get_data()) for d in data_sets])) * len(theta) -2*lnprob_oLCDM(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = 9, 50, 100
pos0 = np.random.uniform(ranges_oLCDM_min, ranges_oLCDM_max, (nwalkers,len(ranges_oLCDM_max)))

# pool = Pool(cpu_count())
# write = HDFBackend('chains/LCDM_' + str(nwalkers) + 'x' + str(nsteps) + '.h5')
# sampler = EnsembleSampler(nwalkers, ndim, lnprob_LCDM, pool=pool)#, backend=write)

# sampler.run_mcmc(pos0, nsteps, progress=True)

## ...or load existing chains

In [ ]:
oLCDM_sampler = HDFBackend('chains/oLCDM' + data_sets_str + '512x1000.h5', read_only=True)
# oLCDM_sampler = sampler

nsteps, nwalkers, ndim = oLCDM_sampler.get_chain().shape

labs = [r'$\Omega_m$', r'$\Omega_\Lambda$', r'$\Omega_b$', r'$H_0$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']

## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 300
while Nmin < nsteps and np.all(oLCDM_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
oLCDM_samples = oLCDM_sampler.get_chain()

f, ax = plt.subplots(len(oLCDM_samples.T), 1, figsize=(6,3*len(oLCDM_samples.T)))

for i in range(len(oLCDM_samples.T)):
    ax[i].plot(oLCDM_samples.T[i].T, lw=0.5)
    ax[i].plot([Nmin, Nmin], [min(oLCDM_samples.T[i].flatten()), max(oLCDM_samples.T[i].flatten())], c='k', lw=1.5)
    ax[i].text(Nmin, min(oLCDM_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
# plt.savefig('plots/convergence_oLCDM.png')
plt.show()

## Plot the 1$\sigma$ and 2$\sigma$ contours

In [ ]:
oLCDM_samples = oLCDM_samples[Nmin:, :].reshape((-1, ndim))


meanoLCDM = np.mean(oLCDM_samples, axis=0)
stdoLCDM = np.var(oLCDM_samples, axis=0)
maxoLCDM=[]
for i in range(len(meanoLCDM)):
    like = np.histogram(oLCDM_samples.T[i], bins=50)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxoLCDM.append(max_val)

fig = corner(oLCDM_samples[:,:4], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)), range=[(0,0.8),(0,1),(0,.1), (60,75)],
             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
#              truths=maxoLCDM)




plt.tight_layout()
# plt.savefig('plots/posterior_oLCDM' + data_sets_str[:-1] + '.pdf')

## Visualize the results

In [ ]:
v = np.percentile(oLCDM_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

omegam, stdmp, stdmm = v[0]
omegac, stdcp, stdcm = v[1]

omegab, stdbp, stdbm = v[2]
H0, stdhp, stdhm = v[3]
    
    
#model
z = SNdata.get_data().T[0]
best_fit_cosmo_oLCDM = cosmology(*maxoLCDM[:2], omegab = maxoLCDM[2], Hzero=maxoLCDM[3])
SNdata.set_param(v.T[0][4:8])
Qdata.set_param(v.T[0][8:])

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])

if 'BAO'  in data_set_names or 'CMB' in data_set_names:    
    ax1.set_title('$\Omega_m = {%1.3f}^{+%1.3f}_{-%1.3f},\, \Omega_\Lambda = {%1.3f}^{+%1.3f}_{-%1.3f},\, H_0 = {%1.1f}^{+%1.1f}_{-%1.1f}$' % (omegam, stdmp,stdmm, omegac, stdcp, stdcm, H0, stdhm, stdhp))
else:
    ax1.set_title('$\Omega_m = {%1.3f}^{+%1.3f}_{-%1.3f},\, \Omega_\Lambda = {%1.3f}^{+%1.3f}_{-%1.3f}$' % (omegam, stdmp,stdmm, omegac, stdcp, stdcm))

if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=0, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(best_fit_cosmo_oLCDM).T[0], BAOdata.distance_modulus(best_fit_cosmo_oLCDM).T[1], yerr=BAOdata.delta_distance_modulus(best_fit_cosmo_oLCDM), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=1, label=r'BAO')



zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_cosmo_oLCDM.distance_modulus(zPlot), c='k', label=r'best fit')
ax1.fill_between(zPlot, cosmology(omegam+stdmp,omegac+stdcp, omegab=omegab+stdbp, Hzero=H0+stdhp).distance_modulus(zPlot), cosmology(omegam-stdmm,omegac-stdcm, omegab=omegab-stdbm, Hzero=H0-stdhm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend()


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_cosmo_oLCDM.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .94, r'$\Delta \mathrm{BIC}(o\Lambda\mathrm{CDM}) = %1.0f$' % (BIC_oLCDM(maxoLCDM) - BIC_oLCDM(theta_oLCDM)), ha='right', va='top', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
# plt.savefig('plots/Hubble_oLCDM' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

In [ ]:
oLCDM_chains = {}
oLCDM_chains['SN'] = HDFBackend('chains/oLCDM_SN_512x1000.h5')
oLCDM_chains['SN_Q'] = HDFBackend('chains/oLCDM_SN_Quasars_512x1000.h5')
oLCDM_chains['SN_BAO_CMB'] = HDFBackend('chains/oLCDM_SN_BAO_CMB_512x1000.h5')
oLCDM_chains['SN_Q_BAO_CMB'] = HDFBackend('chains/oLCDM_SN_Quasars_BAO_CMB_512x1000.h5')
oLCDM_chains['BAO'] = HDFBackend('chains/oLCDM_BAO_512x1000.h5')
oLCDM_chains['CMB'] = HDFBackend('chains/oLCDM_CMB_512x1000.h5')

In [ ]:
Nmin = 300
for sampler in oLCDM_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print('Nmin=',Nmin)

In [ ]:
samples = oLCDM_chains['SN'].get_chain()[Nmin:,:].reshape((-1, oLCDM_chains['SN'].get_chain().shape[-1]))
fig=corner(samples[:,:4], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
            range=[(0,.6), (0,1.25), (0,0.1), (62,75)])

samples = oLCDM_chains['SN_Q'].get_chain()[Nmin:,:].reshape((-1, oLCDM_chains['SN_Q'].get_chain().shape[-1]))
corner(samples[:,:4], levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='blue', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig, range=[(0,.6), (0,1.25), (0,0.1), (62,75)])

samples = oLCDM_chains['BAO'].get_chain()[Nmin:,:].reshape((-1, oLCDM_chains['BAO'].get_chain().shape[-1]))
fig=corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig, range=[(0,.6), (0,1.25), (0,0.1), (62,75)])

samples = oLCDM_chains['CMB'].get_chain()[Nmin:,:].reshape((-1, oLCDM_chains['CMB'].get_chain().shape[-1]))
corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange'], alpha=0.4),
            fig=fig, range=[(0,.6), (0,1.25), (0,0.1), (62,75)])

samples = oLCDM_chains['SN_Q_BAO_CMB'].get_chain()[Nmin:,:].reshape((-1, oLCDM_chains['SN_Q_BAO_CMB'].get_chain().shape[-1]))
corner(samples[:,:4], levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            fig=fig, range=[(0,.6), (0,1.25), (0,0.1), (62,75)])

plt.tight_layout()
# plt.savefig('plots/posterior_oLCDM.pdf')


plt.show()

# Model 2: $w\Lambda$LCDM

## which data sets to include?

In [ ]:
data_sets = [SNdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

## Define the likelihood and priors

In [ ]:
ranges_wLCDM_min=[0, 0, 0, 60., -2.5, -5, -10, -30, -.5, 0, 0]
ranges_wLCDM_max=[1, 1.5, 0.1, 80., -1/3., 5, 10, -10, .5, 10, 3]
def lnprob_wLCDM(theta):
    l = likelihood(theta, data_sets, ranges_wLCDM_min, ranges_wLCDM_max, model = 'wLCDM')
    return l.logprobability_gauss_prior()

## define a reference BIC

In [ ]:
BIC_wLCDM = lambda theta: np.log(sum([len(d.get_data()) for d in data_sets])) * len(theta) -2*lnprob_wLCDM(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = 10, 20, 100
pos0 = np.random.uniform(ranges_wLCDM_min, ranges_wLCDM_max, (nwalkers,len(ranges_wLCDM_max)))

pool = Pool(cpu_count())
write = HDFBackend('chains/LCDM_' + str(nwalkers) + 'x' + str(nsteps) + '.h5')
sampler = EnsembleSampler(nwalkers, ndim, lnprob_wLCDM, pool=pool)#, backend=write)

# tau, accepted = [], []

# # Now we'll sample for up to nsteps steps
# for i, sample in enumerate(sampler.sample(pos0, iterations=nsteps, progress=True)):
#     # Only check convergence every 10 steps
#     if sampler.iteration % 10:
#         continue

#     # Compute the autocorrelation time so far
#     # Using tol=0 means that we'll always get an estimate even
#     # if it isn't trustworthy
#     tau.append(np.mean(sampler.get_autocorr_time(tol=0)))
#     accepted.append(np.mean(sampler.acceptance_fraction))
# pool.close()

## ...or load existing chains

In [ ]:
wLCDM_sampler = HDFBackend('chains/wLCDM' + data_sets_str + '512x1000.h5', read_only=True)
#LCDM_sampler = sampler

nsteps, nwalkers, ndim = wLCDM_sampler.get_chain().shape

labs = [r'$\Omega_m$', r'$\Omega_\Lambda$', r'$H_0$', r'$r_s$', r'$w$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']

## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 300
while Nmin < nsteps and np.all(wLCDM_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
wLCDM_samples = wLCDM_sampler.get_chain()

f, ax = plt.subplots(len(wLCDM_samples.T), 1, figsize=(6,3*len(wLCDM_samples.T)))

for i in range(len(wLCDM_samples.T)):
    ax[i].plot(wLCDM_samples.T[i].T, lw=0.5)
    ax[i].plot([Nmin, Nmin], [min(wLCDM_samples.T[i].flatten()), max(wLCDM_samples.T[i].flatten())], c='k', lw=1.5)
    ax[i].text(Nmin, min(wLCDM_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
# plt.savefig('plots/convergence_wLCDM.png')
plt.show()

## Plot the 1$\sigma$ and 2$\sigma$ contours

In [ ]:
wLCDM_samples = wLCDM_samples[Nmin:, :].reshape((-1, ndim))


meanwLCDM = np.mean(wLCDM_samples, axis=0)
stdwLCDM = np.var(wLCDM_samples, axis=0)
maxwLCDM=[]
for i in range(len(meanwLCDM)):
    like = np.histogram(wLCDM_samples.T[i], bins=50)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxwLCDM.append(max_val)

fig = corner(wLCDM_samples[:,:5], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
# #              truths=maxwLCDM)




plt.tight_layout()
# plt.savefig('plots/posterior_wLCDM' + data_sets_str[:-1] + '.pdf')

## Visualize the results

In [ ]:
v = np.percentile(wLCDM_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

omegam, stdmp, stdmm = v[0]
omegac, stdcp, stdcm = v[1]
w, wp, wm = v[4]

if 'BAO' in data_set_names:
    H0, stdhp, stdhm = v[2]
    rs, stdrsp, stdrsm = v[3]
else:
    H0, stdhp, stdhm = 70., 0, 0
    rs, stdrsp, stdrsm = 147.27, 0, 0

#model
z = SNdata.get_data().T[0]
# omegam, omegac, w = maxwLCDM[0], maxwLCDM[1], maxwLCDM[2]
best_fit_cosmo = cosmology(omegam, omegac, w=w,rs=rs,Hzero=H0)
SNdata.set_param(v.T[0][-6:-2])
Qdata.set_param(v.T[0][-2:])
BAOdata.set_param([omega_baryon_preset, omega_gamma_preset])

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])
ax1.set_title(r'$\Omega_m = {%1.2f}^{+%1.2f}_{-%1.2f},\, \Omega_\Lambda = {%1.2f}^{+%1.2f}_{-%1.2f},\, w = {%1.2f}^{+%1.2f}_{-%1.2f},\, H_0 =  {%1.2f}^{+%1.2f}_{-%1.2f}$' % (omegam, stdmp,stdmm, omegac, stdcp,stdcm, w, wp, wm, H0, stdhp,stdhm))#, stdkm, stdkp))

if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=-1, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=-1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(best_fit_cosmo_LCDM).T[0], BAOdata.distance_modulus(best_fit_cosmo_LCDM).T[1], yerr=BAOdata.delta_distance_modulus(best_fit_cosmo_LCDM), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=1, label=r'BAO')



zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_cosmo.distance_modulus(zPlot), c='k', label=r'fit')
ax1.fill_between(zPlot, cosmology(omegam+stdmp,omegac+stdcp, w=w + wp, rs = rs+stdrsp, Hzero=H0+stdhp).distance_modulus(zPlot), cosmology(omegam-stdmm,omegac-stdcm,w=w - wm, rs = rs-stdrsm, Hzero=H0-stdhm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend()


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_cosmo_LCDM.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .94, r'$\Delta\mathrm{BIC}(w\Lambda\mathrm{CDM}) = %1.0f$' % (BIC_wLCDM(maxwLCDM) - BIC_LCDM(theta_LCDM)), ha='right', va='top', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
# plt.savefig('plots/Hubble_wLCDM' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

In [ ]:
wLCDM_chains = {}
wLCDM_chains['SN'] = HDFBackend('chains/wLCDM_SN_512x1000.h5')
wLCDM_chains['SN_Q'] = HDFBackend('chains/wLCDM_SN_Quasars_512x1000.h5')
wLCDM_chains['SN_Q_BAO'] = HDFBackend('chains/wLCDM_SN_Quasars_BAO_512x1000.h5')
wLCDM_chains['BAO'] = HDFBackend('chains/wLCDM_BAO_512x1000.h5')

In [ ]:
Nmin = 300
for sampler in wLCDM_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print('Nmin=',Nmin)

In [ ]:
samples = wLCDM_chains['SN'].get_chain()[Nmin:,:].reshape((-1, wLCDM_chains['SN'].get_chain().shape[-1]))
fig=corner(samples[:,:5], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
                labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
                fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']))#,
#                 range=[(0,1), (0,1.6), (-2,-.5)])

samples = wLCDM_chains['SN_Q'].get_chain()[Nmin:,:].reshape((-1, wLCDM_chains['SN_Q'].get_chain().shape[-1]))
fig=corner(samples[:,:5], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig)#, range=[(0,1), (0,1.6), (-2,-.5)])


samples = wLCDM_chains['SN_Q_BAO'].get_chain()[Nmin:,:].reshape((-1, wLCDM_chains['SN_Q_BAO'].get_chain().shape[-1]))
corner(samples[:,:5], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig)#, range=[(0,1), (0,1.6), (-2,-.5)])

# samples = wLCDM_chains['BAO'].get_chain()[Nmin:,:].reshape((-1, wLCDM_chains['SN_Q_BAO'].get_chain().shape[-1]))
# corner(samples[:,:5], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgray', 'gray'], alpha=0.4),
#             fig=fig)#, range=[(0,1), (0,1.6), (-2,-.5)])



plt.tight_layout()
# plt.savefig('plots/posterior_wLCDM.pdf')

plt.show()

# Model 3: Bigravity

## which data sets to include?

In [ ]:
data_sets = [SNdata, Qdata, BAOdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

## Define the likelihood and priors

In [ ]:
ranges_bigravity_min=[-33., 0, 0, 0, 60, -5, -10, -30, -.5, 0, 0]
ranges_bigravity_max=[-28., np.pi/2., 1, .1, 80, 5, 10, -10, .5, 10, 3]
def lnprob_bigravity(theta):
    l = likelihood(theta, data_sets, ranges_bigravity_min, ranges_bigravity_max, model = 'bigravity')
    return l.logprobability_gauss_prior()

## define a reference BIC

In [ ]:
BIC_bigravity = lambda theta: np.log(sum([len(d.get_data()) for d in data_sets])) * len(theta) -2*lnprob_bigravity(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = len(ranges_bigravity_min), 24, 100
pos0 = np.random.uniform(ranges_bigravity_min, ranges_bigravity_max, (nwalkers,len(ranges_bigravity_max)))


pool = Pool(cpu_count())
write = HDFBackend('chains/bigravity_' + str(nwalkers) + 'x' + str(nsteps) + '.h5')
sampler = EnsembleSampler(nwalkers, ndim, lnprob_bigravity, pool=pool)#, backend=write)

sampler.run_mcmc(pos0, nsteps, progress=True)

## ...or load existing chains

In [ ]:
bigravity_sampler = HDFBackend('chains/bigravity' + data_sets_str + '512x1000.h5', read_only=True)
#LCDM_sampler = sampler

nsteps, nwalkers, ndim = bigravity_sampler.get_chain().shape

if ndim == 9:
    labs = [r'$\log_{10} (m_g/\mathrm{eV})$', r'$\theta$', r'$\Omega_m$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']
elif ndim >9:
    labs = np.insert(labs, 2, [r'$\beta_0$', r'$\beta_1$', r'$\beta_2$', r'$\beta_3$'])

## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 300
while Nmin < nsteps and np.all(bigravity_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
best_fit_bigravitycosmo.get_energy_densities()

In [ ]:
bigravity_samples = bigravity_sampler.get_chain()

f, ax = plt.subplots(len(bigravity_samples.T), 1, figsize=(6,3*len(bigravity_samples.T)))

for i in range(len(bigravity_samples.T)):
    ax[i].plot(bigravity_samples.T[i].T, lw=0.5)
    ax[i].plot([Nmin, Nmin], [min(bigravity_samples.T[i].flatten()), max(bigravity_samples.T[i].flatten())], c='k', lw=1.5)
    ax[i].text(Nmin, min(bigravity_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
# plt.savefig('plots/convergence_bigravity.png')
plt.show()

## Plot the 1$\sigma$ and 2$\sigma$ contours

In [ ]:
bigravity_samples = bigravity_samples[Nmin:, :].reshape((-1, ndim))


meanbigravity = np.mean(bigravity_samples, axis=0)
stdbigravity = np.var(bigravity_samples, axis=0)
maxbigravity=[]
for i in range(len(meanbigravity)):
    like = np.histogram(bigravity_samples.T[i], bins=50)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxbigravity.append(max_val)

fig = corner(bigravity_samples[:,:3], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
# #              truths=maxbigravity)




plt.tight_layout()
# plt.savefig('plots/posterior_bigravity' + data_sets_str[:-1] + '.pdf')

## Visualize the results

In [ ]:
v = np.percentile(bigravity_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

log10m, stdmp, stdmm = v[0]
theta, stdtp, stdtm = v[1]
if ndim == 9:
    b0, b1, b2, b3 = 1.,1.,-1.,1.
elif ndim >=9:
    b0, b1, b2, b3 = v[2:6].T[0]

omegam, stdmp, stdmm = v[-7]

#model
z = SNdata.get_data().T[0]
best_fit_bigravitycosmo = bigravity_cosmology(maxbigravity[0], maxbigravity[1], b0, b1, b2, b3, maxbigravity[-7])
SNdata.set_param(v.T[0][-6:-2])
Qdata.set_param(v.T[0][-2:])
BAOdata.set_param([omega_baryon_preset, omega_gamma_preset])

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])
ax1.set_title(r'$\Omega_m = {%1.2f}^{+%1.2f}_{-%1.2f},\, m_g = 10^{{%1.2f}^{+%1.2f}_{-%1.2f}}\mathrm{eV},\, \theta = {%1.2f}^{+%1.2f}_{-%1.2f}$' % (omegam, stdmp, stdmm, log10m, stdmm, stdmp, theta, stdtm, stdtp))#, stdkm, stdkp))

if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=-1, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=-1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(cosmology(Omega_m_preset, Omega_c_preset, Omega_r_preset)).T[0], BAOdata.distance_modulus(cosmology(Omega_m_preset, Omega_c_preset, Omega_r_preset)).T[1], yerr=BAOdata.delta_distance_modulus(cosmology(Omega_m_preset, Omega_c_preset, Omega_r_preset)), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=-1, label=r'BAO')



zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_bigravitycosmo.distance_modulus(zPlot), c='k', label=r'fit')
ax1.fill_between(zPlot, bigravity_cosmology(log10m+stdmp, theta + stdtp, b0,b1,b2,b3, omegam + stdmp).distance_modulus(zPlot), bigravity_cosmology(log10m-stdmm, theta - stdtm, b0,b1,b2,b3, omegam - stdmm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend()


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_bigravitycosmo.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .94, r'$\Delta \mathrm{BIC}(\mathrm{bigravity}) = %1.0f$' % (BIC_bigravity(maxbigravity) - BIC_LCDM(theta_LCDM)), ha='right', va='top', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
# plt.savefig('plots/Hubble_bigravity' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

In [ ]:
bigravity_chains = {}
bigravity_chains['SN'] = HDFBackend('chains/bigravity_SN_512x1000.h5')
bigravity_chains['SN_Q'] = HDFBackend('chains/bigravity_SN_Quasars_512x1000.h5')
# bigravity_chains['SN_Q_BAO'] = HDFBackend('chains/bigravity_SN_Quasars_BAO_512x1000.h5')

In [ ]:
Nmin = 300
for sampler in bigravity_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print("Nmin = ", Nmin)

In [ ]:
samples = bigravity_samples#bigravity_chains['SN'].get_chain()[Nmin:,:].reshape((-1, bigravity_chains['SN'].get_chain().shape[-1]))
fig=corner(samples, quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
                labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
                fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']))
#                 range=[(0,1), (0,1.3), (-2,-.5)])

# samples = bigravity_chains['SN_Q'].get_chain()[Nmin:,:].reshape((-1, bigravity_chains['SN_Q'].get_chain().shape[-1]))
# corner(samples[:,:3], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6))
# #             fig=fig, range=[(0,1), (0,1.3), (-2,-.5)])


# samples = bigravity_chains['SN_Q_BAO'].get_chain()[Nmin:,:].reshape((-1, bigravity_chains['SN_Q_BAO'].get_chain().shape[-1]))
# corner(samples[:,:3], quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
#             fig=fig, range=[(0,1), (0,1.3), (-2,-.5)])




plt.tight_layout()
# plt.savefig('plots/posterior_bigravity.pdf')

plt.show()

# Model 4: Conformal Gravity

## which data sets to include?

In [ ]:
data_sets = [SNdata, Qdata, BAOdata]
data_set_names = [set.name for set in data_sets]
data_sets_str = '_' + '_'.join([name for name in data_set_names]) + '_' 

## Define the likelihood and priors

In [ ]:
ranges_conformal_min=[0, 50, 0, 60, -5, -10, -30, -.5, 0, 0]
ranges_conformal_max=[1, 300, 0.1, 80, 5, 10, -10, .5, 10, 3]
def lnprob_conformal(theta):
    l = likelihood(theta, data_sets, ranges_conformal_min, ranges_conformal_max, model = 'conformal')
    return l.logprobability_gauss_prior()

## define a reference BIC

In [ ]:
BIC_conformal = lambda theta: np.log(sum([len(d.get_data()) for d in data_sets])) * len(theta) -2*lnprob_conformal(theta)

## run the MCMC sampler  [arXiv:1202.3665]...

In [ ]:
ndim, nwalkers, nsteps = len(ranges_conformal_min), 20, 100
pos0 = np.random.uniform(ranges_conformal_min, ranges_conformal_max, (nwalkers,len(ranges_conformal_max)))

# pool = Pool(cpu_count())
# write = HDFBackend('chains/conformal_' + str(nwalkers) + 'x' + str(nsteps) + '.h5')
# sampler = EnsembleSampler(nwalkers, ndim, lnprob_conformal, pool=pool)#, backend=write)

# sampler.run_mcmc(pos0, nsteps, progress=True)

## ...or load existing chains

In [ ]:
conformal_sampler = HDFBackend('chains/conformal' + data_sets_str + '512x1000.h5', read_only=True)
#LCDM_sampler = sampler

nsteps, nwalkers, ndim = conformal_sampler.get_chain().shape

labs=[r'$(\gamma_0/\mathrm{Gpc}^{-1})$', r'$(\kappa/\mathrm{Gpc}^{-2})$', r'$\Omega_b$', r'$H_0$', r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']

## Check and visualise the convergence: $\tau_f / n_\mathrm{steps} < 1/30$

In [ ]:
Nmin = 500
while Nmin < nsteps and np.all(conformal_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
conformal_samples = conformal_sampler.get_chain()
conformal_samples = conformal_samples[:,np.where(conformal_samples[0].T[0] < .33)[0],:]

f, ax = plt.subplots(len(conformal_samples.T), 1, figsize=(6,3*len(conformal_samples.T)))

for i in range(len(conformal_samples.T)):
    ax[i].plot(conformal_samples.T[i].T, lw=0.5)
    ax[i].plot([Nmin, Nmin], [min(conformal_samples.T[i].flatten()), max(conformal_samples.T[i].flatten())], c='k', lw=1.5)
    ax[i].text(Nmin, min(conformal_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
# plt.savefig('plots/convergence_conformal.png')
plt.show()

## Plot the 1$\sigma$ and 2$\sigma$ contours

In [ ]:
conformal_samples = conformal_samples[Nmin:, :].reshape((-1, ndim))
conformal_samples = conformal_samples[conformal_samples.T[0]**2 < .33]

meanconformal = np.mean(conformal_samples, axis=0)
stdconformal = np.var(conformal_samples, axis=0)
maxconformal=[]
for i in range(len(meanconformal)):
    like = np.histogram(conformal_samples.T[i], bins=50)
    i_max=np.argmax(like[0])
    max_val = (like[1][i_max]+like[1][i_max+1])/2
    maxconformal.append(max_val)

fig = corner(conformal_samples, quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'))#,
# #              truths=maxconformal)




plt.tight_layout()
# plt.savefig('plots/posterior_conformal' + data_sets_str[:-1] + '.pdf')

In [ ]:
v = np.percentile(conformal_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

gamma0, gamm0p, gamma0m = v[0]
kappa, kappap, kappam = v[1]

omegak =  (gamma0)**2 / 2 * cosmology.cLight**2/(70./1E-3)**2#Gpc^-1
stdkp, stdkm = np.array([gamm0p, gamma0m]) * (gamma0) * cosmology.cLight**2/(70./1E-3)**2#Gpc^-1


omegab, stdbp, stdbm = v[2]
H0, stdhp, stdhm = v[3]

omegac = 1 - omegak
stdcp = stdkm
stdcm = stdkp
#model
z = SNdata.get_data().T[0]

best_fit_conformalcosmo = cosmology(0, omegac, omegab=maxconformal[2], Hzero=maxconformal[3])
SNdata.set_param(v.T[0][2:6])
Qdata.set_param(v.T[0][6:8])

In [ ]:
np.round(v[0],3)

In [ ]:
np.round(v[2],3)

In [ ]:
np.round(v[3],3)

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])
ax1.set_title(r'$\Omega_\Lambda = {%1.3f}^{+%1.3f}_{-%1.3f},\, \Omega_k = {%1.3f}^{+%1.3f}_{-%1.3f}$' % (omegac, stdcp,stdcm, omegak, stdkm, stdkp))

if 'Quasars' in data_set_names:
    ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=-1, label=r'quasars')
if 'SN' in data_set_names:
    ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=SNdata.delta_distance_modulus(), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=-1, label=r'SNe')
if 'BAO' in data_set_names:
    ax1.errorbar(BAOdata.distance_modulus(best_fit_conformalcosmo).T[0], BAOdata.distance_modulus(best_fit_conformalcosmo).T[1], yerr=BAOdata.delta_distance_modulus(best_fit_conformalcosmo), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=1, label=r'BAO')



zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_conformalcosmo.distance_modulus(zPlot), c='k', label=r'fit')
ax1.fill_between(zPlot, cosmology(0,omegac+stdcp, omegab=omegab+stdbp, Hzero=H0 + stdhp).distance_modulus(zPlot), cosmology(0,omegac-stdcm, omegab=omegab-stdbm, Hzero=H0 - stdhm).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

ax1.set_xlim(0.01,6)
ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend()


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_conformalcosmo.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.97, .94, r'$\Delta\mathrm{BIC}(\mathrm{CG}) = %1.0f$' % (BIC_conformal(maxconformal) - BIC_oLCDM(maxoLCDM)), ha='right', va='top', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
plt.savefig('plots/Hubble_conformal' + data_sets_str[:-1] + '.pdf')
plt.show()

## Joint fit of all data sets

In [ ]:
conformal_chains = {}
conformal_chains['SN'] = HDFBackend('chains/conformal_SN_512x1000.h5')
conformal_chains['SN_Q'] = HDFBackend('chains/conformal_SN_Quasars_512x1000.h5')
conformal_chains['BAO'] = HDFBackend('chains/conformal_BAO_512x1000.h5')
conformal_chains['CMB'] = HDFBackend('chains/conformal_CMB_512x1000.h5')
conformal_chains['SN_Q_BAO'] = HDFBackend('chains/conformal_SN_Quasars_BAO_512x1000.h5')

In [ ]:
Nmin = 600
for sampler in conformal_chains.values():
    nsteps, nwalkers, ndim = sampler.get_chain().shape
    while Nmin < nsteps and np.all(sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*30 > 1):
        Nmin+=100
print('Nmin=',Nmin)

### No Rot curves -> exclude $\kappa$

In [ ]:
labs = np.delete(labs, 1)

In [ ]:
samples = conformal_chains['SN'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['SN'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples = np.array([samples.T[i] for i in (0,2,3)]).T
fig=corner(samples, levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='green',
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightgreen', 'green']),
            range=[(0,1), (0, .1), (62,75)])


samples = conformal_chains['SN_Q'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['SN_Q'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples = np.array([samples.T[i] for i in (0,2,3)]).T
corner(samples, quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)), color='blue',
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'blue'], alpha=0.6),
            fig=fig, range=[(0,1), (0, .1), (62,75)])


samples = conformal_chains['BAO'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['BAO'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples = np.array([samples.T[i] for i in (0,2,3)]).T
fig=corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='red', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'pink', 'red'], alpha=0.4),
            fig=fig, range=[(0,1), (0, .1), (62,75)])


# samples = conformal_chains['CMB'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['CMB'].get_chain().shape[-1]))
# samples = samples[np.where(samples.T[0] < .33)[0]]
# corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
#             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='orange', 
#             fill_contours=True, contourf_kwargs=dict(colors=['white', 'yellow', 'orange'], alpha=0.4),
#             fig=fig)#, range=[(0,1), (50,300), (0, .1), (62,75)])

samples = conformal_chains['SN_Q_BAO'].get_chain()[Nmin:,:].reshape((-1, conformal_chains['SN_Q_BAO'].get_chain().shape[-1]))
samples = samples[np.where(samples.T[0] < .33)[0]]
samples = np.array([samples.T[i] for i in (0,2,3)]).T
corner(samples, levels=(1-np.exp(-0.5*1),1-np.exp(-0.5*4)),
            labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, color='cyan', 
            fill_contours=True, contourf_kwargs=dict(colors=['white', 'lightblue', 'cyan'], alpha=0.4),
            fig=fig, range=[(0,1), (0, .1), (62,75)])



plt.tight_layout()
# plt.savefig('plots/posterior_conformal.pdf')

plt.show()

## Rotation curve data

In [ ]:
log10gamma0, log10kappa0 = -6, -4
RCdata = RC_data([10**log10gamma0, 10**log10kappa0])

In [ ]:
#likelihood function
def lnlikeCG(theta, SNdata, Qdata, RCdata, BAOdata):
    Omegam, gamma0, kappa, a, b, MB, delta_Mhost, beta_prime, s = theta
    
    Omegak =  (gamma0)**2 / 2 *cosmology.cLight**2/(70/1E-3)**2
    Omegac = 1-Omegam-Omegak
    
    LCDM = cosmology(Omegam, Omegac)
    SNdata.set_param(np.array([a,b,MB, delta_Mhost]))
    Qdata.set_param(np.array([beta_prime, s]))
    
    log_prob = LCDM.log_likelihood(SNdata)
    log_prob += LCDM.log_likelihood(Qdata)
    log_prob += LCDM.log_likelihood(BAOdata)
    
    if np.isnan(log_prob):
        return -np.inf

    RCdata.set_param([gamma0, kappa])
    RCdata.fit()
    log_prob += RCdata.get_loglike()


    return log_prob


def lnpriorCG(theta):
    Omegam, gamma0, kappa, a, b, MB, delta_Mhost, beta_prime, s = theta
    
    Omegak =  (gamma0)**2 / 2 *cosmology.cLight**2/(70./1E-3)**2
    Omegac = 1-Omegam-Omegak

    if -.1 < Omegam < 0 and 0 < Omegak < 1.0 and gamma0 > 0 and 0 < kappa < 1000 and -5 <= a < 5 and -10 < b < 10 and -50 < MB < 0 and -0.5 < delta_Mhost < 0.5 and 0 < s <= 3:
        return 0.0
    return -np.inf

def lnprobCG(theta, SNdata, Qdata, RCdata):
    lp = lnpriorCG(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlikeCG(theta, SNdata, Qdata, RCdata)


## Load Conformal Gravity data

In [ ]:
CG_sampler = HDFBackend('chains/CG_nonlog_500x1000.h5', read_only=True)

nsteps, nwalkers, ndim = CG_sampler.get_chain().shape
labs=[r'$\Omega_m$', r'$(\gamma_0/\mathrm{Gpc}^{-1})$', r'$(\kappa/\mathrm{Gpc}^{-2})$',r'$\alpha$', r'$\beta$', r'$M_B$', r'$\Delta M_B$', r'$\beta^\prime$', r'$\delta$']

In [ ]:
Nmin = 0
while Nmin < nsteps and np.all(CG_sampler.get_autocorr_time(tol=0,discard=Nmin)/(nsteps)*50 > 1):
    Nmin+=100
    
print('Nmin = {}'.format(Nmin))

In [ ]:
CG_samples = CG_sampler.get_chain()

f, ax = plt.subplots(len(CG_samples.T), 1, figsize=(6,3*len(CG_samples.T)))

for i in range(len(CG_samples.T)):
    ax[i].plot(CG_samples.T[i].T, lw=0.5)
    ax[i].plot([Nmin, Nmin], [min(CG_samples.T[i].flatten()), max(CG_samples.T[i].flatten())], c='k', lw=1.5)
    ax[i].text(Nmin, min(CG_samples.T[i].flatten()), r'$n_\mathrm{min}$', rotation=90, ha='right')
    ax[i].set_ylabel(labs[i])
    ax[i].set_xlabel('iteration')
    
    
plt.tight_layout()
# plt.savefig('plots/convergence_CG.png')
plt.show()

In [ ]:
CG_samples = CG_sampler.get_chain()[Nmin:, :, :].reshape((-1, ndim))

#plot Omega_k instead of gamma0
CG_samples[:,1] = CG_samples[:,1]**2 / 2 *cosmology.cLight**2/(70./1E-3)**2
labs[1] = r'$\Omega_k$'


meanCG = np.mean(CG_samples, axis=0)
stdCG = np.var(CG_samples, axis=0)
maxCG=[]
for i in range(len(meanCG)):
    likelihood = np.histogram(CG_samples.T[i], bins=50)
    i_max=np.argmax(likelihood[0])
    max_val = (likelihood[1][i_max]+likelihood[1][i_max+1])/2
    maxCG.append(max_val)

fig = corner(CG_samples, quantiles=(.16,.84),  levels=(1-np.exp(-0.5),1-np.exp(-0.5*4)),# range=[(-.005, 0), (.0222,.0226), (71,73)],
             labels=labs, smooth=True, smooth1d=True, bins=25, plot_datapoints=False, fill_contours=True, contourf_kwargs=dict(colors=None, cmap='Greens'),
             truths=maxCG)

plt.tight_layout()
# plt.savefig('plots/posterior_CG.pdf')

## Visualize the results

In [ ]:
v = np.percentile(CG_samples,[16, 50, 84], axis = 0)
v = np.asarray([v[1], v[2]-v[1],v[1]-v[0]]).T

omegam, stdmp, stdmm = v[0]
omegak, stdkp, stdkm  = v[1]

#model
z = SNdata.get_data().T[0]
best_fit_cosmo = cosmology(omegam, 1- omegam - omegak)
SNdata.set_param(maxCG[3:7])
Qdata.set_param(maxCG[7:])
RC_data.set_param(maxCG[1:3])

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0)


ax1 = plt.subplot(gs[0:2, 0])
ax1.set_title(r'$\Omega_m = ({%1.2f}^{+%1.2f}_{-%1.2f})\cdot 10^{-3},\, \Omega_k = ({%1.2f}^{+%1.2f}_{-%1.2f})\cdot 10^{-4},\, \kappa = {%1.1f}^{+%1.1f}_{-%1.1f}\, \mathrm{Gpc}^{-2}$' % (1E3*omegam, 1E3*stdmp,1E3*stdmm, 1E4*omegak, 1E4*stdkp,1E4*stdkm, *v[2]), fontsize=16)#, stdkm, stdkp))

ax1.errorbar(Qdata.distance_modulus().T[0], Qdata.distance_modulus().T[1], yerr=Qdata.delta_distance_modulus(), linestyle='none', marker='o', color='orange', ecolor='yellow', markersize=3, alpha=0.6, zorder=-1, label=r'quasars')
ax1.errorbar(SNdata.distance_modulus().T[0], SNdata.distance_modulus().T[1], yerr=np.diag(SNdata.delta_distance_modulus()), linestyle='none', marker='o', color='cyan', ecolor='blue', markersize=3, alpha=0.6, zorder=-1, label=r'SNe')
ax1.errorbar(BAOdata.distance_modulus(cosmology(Omega_m_preset, Omega_c_preset, Omega_r_preset)).T[0], BAOdata.distance_modulus(cosmology(Omega_m_preset, Omega_c_preset, Omega_r_preset)).T[1], yerr=BAOdata.delta_distance_modulus(cosmology(Omega_m_preset, Omega_c_preset, Omega_r_preset)), linestyle='none', marker='o', color='red', ecolor='black', markersize=3, alpha=0.6, zorder=-1, label=r'BAO')





zPlot = np.logspace(-3,np.log10(6),100)
ax1.plot(zPlot, best_fit_cosmo.distance_modulus(zPlot), c='k', label=r'fit')
ax1.fill_between(zPlot, cosmology(omegam+stdmp,1- (omegam - stdmm) - (omegak-stdkm)).distance_modulus(zPlot), cosmology(omegam-stdmm,1- (omegam + stdmp) - (omegak+stdkp)).distance_modulus(zPlot), color='gray', alpha=0.3)
ax1.plot(zPlot, cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), ls = '--', c='gray', label=r'$\Lambda$CDM')

# ax1.set_xlim(0.01,6)
# ax1.set_ylim(32,55)


ax1.set_ylabel(r'distance modulus $\mu$')
ax1.set_xticklabels([])

ax1.grid('--', dashes=(5,5))
ax1.legend()


ax2 = plt.subplot(gs[2, 0])

ax2.plot(zPlot, best_fit_cosmo.distance_modulus(zPlot) - cosmology(Omega_m_preset, Omega_c_preset).distance_modulus(zPlot), 'k')

# ax2.set_xscale('log')
ax2.set_xlim(0.01,6)

ax2.text(0.04, .94, r'$\mathrm{BIC}(\mathrm{CG}) = %1.0f$' % (np.log(len(SNdata.get_data()) + len(Qdata.get_data()))*ndim-2*(lnlikeCG(maxCG,SNdata,Qdata,RC_data,BAOdata) - RC_data.get_loglike())), ha='left', va='top', transform=ax2.transAxes, bbox=dict(facecolor=(1,1,1,.8), edgecolor='lightgray', boxstyle='round'))


ax2.grid('--', dashes=(5,5))
ax2.set_xlabel(r'$z$')
ax2.set_ylabel(r'$\mu_\mathrm{fit} - \mu_{\Lambda\mathrm{CDM}}$')

plt.tight_layout()
# plt.savefig('plots/Hubble_CG.pdf')
plt.show()